# **Importing libraries and loading data**

In [0]:
#!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix_road_signs'

/content/drive/My Drive/Colab Notebooks/dw_matrix_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

# **Convolutional neural network**

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%M%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )
  
  return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

#get_cnn_v4(input_shape, num_classes).summary()

In [7]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 47ms/step - loss: 2.5090 - accuracy: 0.2874 - val_loss: 1.0376 - val_accuracy: 0.7003
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 0.8128 - accuracy: 0.7382 - val_loss: 0.2420 - val_accuracy: 0.9251
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.3638 - accuracy: 0.8838 - val_loss: 0.1215 - val_accuracy: 0.9667
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2428 - accuracy: 0.9268 - val_loss: 0.0656 - val_accuracy: 0.9791
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.1924 - accuracy: 0.9405 - val_loss: 0.0416 - val_accuracy: 0.9881


0.9714285714285714

# **Hyperopt**

In [8]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.1548 - accuracy: 0.9714


0.9714285731315613

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose=0,
  )
  
  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20)
}

In [10]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.7629 - accuracy: 0.7500

  7/138 [>.............................]
 - ETA: 0s - loss: 0.1231 - accuracy: 0.9598

 12/138 [=>............................]
 - ETA: 1s - loss: 0.1136 - accuracy: 0.9557

 19/138 [===>..........................]
 - ETA: 1s - loss: 0.2510 - accuracy: 0.9260

 25/138 [====>.........................]
 - ETA: 0s - loss: 0.4091 - accuracy: 0.9025

 29/138 [=====>........................]
 - ETA: 1s - loss: 0.3532 - accuracy: 0.9159


# **More detailed optimization**

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

In [15]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3673199978399066, 'dropout_cnn_block_three': 0.4269546056136921, 'dropout_cnn_block_two': 0.3553521043456578, 'dropout_dense_block_one': 0.5926480472322768, 'dropout_dense_block_two': 0.425139294036324}
accuracy=0.9131519198417664
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3001180301959782, 'dropout_cnn_block_three': 0.3805809946542022, 'dropout_cnn_block_two': 0.39726194077659954, 'dropout_dense_block_one': 0.6193908769698965, 'dropout_dense_block_two': 0.39185374138714535}
accuracy=0.940362811088562
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.407855352627824, 'dropout_cnn_block_three': 0.3760770218574634, 'dropout_cnn_block_two': 0.33685383042747546, 'dropout_dense_block_one': 0.5807086726002673, 'dropout_dense_block_two': 0.6734939306577797}
accuracy=0.8512471914291382
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.3671827047223399, 'dropout_cnn_block_three': 0.3310318170535151, 'dropout_cnn_block_two': 0.3449300252808125